# Demo and Validation

In [1]:
from zotbin.binned import *

In [12]:
from tomo_challenge import load_data, load_redshift
from tomo_challenge.jax_metrics import ell_binning, compute_scores
from tomo_challenge.classifiers.random_forest import RandomForest

Load the initialization data:

In [4]:
zedges, ell, ngals, noise, cl_in = load_binned('binned_28.npz')

Specify the challenge data to load:

In [7]:
bands='riz'
include_colors=True
include_errors=True

Load the training data:

In [9]:
training_file='/media/data2/tomo_challenge_data/ugrizy/training.hdf5'
train_data_arr = load_data(training_file, bands, 
                           errors=include_errors,
                           colors=include_errors, array=True)
training_z = load_redshift(training_file)
print(f'Loaded {len(train_data_arr)} training rows.')

/home/dkirkby/DESC/tomo/tomo_challenge/data.py:89: UserWarning: Setting inf (undetected) bands to mag=30
  warnings.warn("Setting inf (undetected) bands to mag=30")


Loaded 8615613 training rows.


Load the validation data:

In [10]:
validation_file='/media/data2/tomo_challenge_data/ugrizy/validation.hdf5'
valid_data_arr = load_data(validation_file, bands, 
                           errors=include_errors,
                           colors=include_errors, array=True)
val_z = load_redshift(validation_file)
print(f'Loaded {len(valid_data_arr)} validation rows.')

Loaded 17228554 validation rows.


Initialize a random forest classifier:

In [11]:
nbins_rf = 4
ntrain_rf = 20000
nvalid_rf = 50000

In [13]:
classifier = RandomForest(bands, {'bins': nbins_rf})

Train on a fraction of the data:

In [14]:
classifier.train(train_data_arr[:ntrain_rf], training_z[:ntrain_rf])

Finding bins for training data
Fitting classifier


/opt/anaconda3/envs/ML/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Test on a fraction of the validation data:

In [15]:
idx_rf = classifier.apply(valid_data_arr[:nvalid_rf])
z_rf = val_z[:nvalid_rf]

Calculate scores using the jax_cosmo implementation provided with tomo_challenge:

In [16]:
%time scores_rf = compute_scores(idx_rf, z_rf, metrics=['SNR_3x2', 'FOM_3x2', 'FOM_DETF_3x2'])

NameError: name 'jc_compute_scores' is not defined